# A Notebook to get a Pawpularity of Your Pet

I saw someone say that they want to measure a Pawpularity of their own pet. This is a brief tutorial.

In [ ]:
%%bash
your_pet="https://pbs.twimg.com/media/FJJXtIQVgAEaDoZ?format=jpg&name=large"

curl -o img ${your_pet}

In [ ]:
from PIL import ImageFont, ImageDraw, Image
img = Image.open('img')
img = img.resize((384, int(img.size[1] / img.size[0] * 384)))
img.save('target.jpg')

In [ ]:
import pandas as pd

#df = pd.DataFrame()
#df['Id'] = ['abcde']
df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv', nrows=1)
df['path'] = ['target.jpg']
df.to_csv('target.csv', index=False)

In [ ]:

import sys
import gc
from tqdm.auto import tqdm
sys.path = ["../input/pytorch-1-10-1/"] + sys.path
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
sys.path.append('../input/convnext/ConvNeXt')
import models.convnext
import models.convnext_isotropic

from fastai.vision.all import *
print(torch.__version__)

BATCH_SIZE = 16

dataset_path = Path('../input/petfinder-pawpularity-score/')

torch.backends.cudnn.benchmark = True

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

test_df = pd.read_csv('target.csv')


def get_data(img_size):
    dls = ImageDataLoaders.from_df(test_df, #pass in train DataFrame
                               #valid_col='is_valid', #
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               #label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(img_size), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls



model_weights = {
    'exp20_convnext_large': 1,
    
 #'exp15_vit_base_patch16_224_miil_in21k': 0.08459557519051042,
 #'exp15_vit_large_patch16_224': 0.054602278608471444,
 #'exp20_convnext_large': 0.10757113794966554,
 #'exp4_crossvit_18_dagger_408': 0.07255291221705243,
 #'exp4_xcit_small_24_p16_384_dist': 0.07615822693896708,
 #'exp4_xcit_small_24_p8_384_dist': 0.07345531269210354,
 #'exp7_cait_m36_384': 0.08676476831055598,
 #'exp7_jx_nest_base': 0.05555615759973762,
 #'exp7_swin_base_patch4_window12_384': 0.0573931578726199,
 #'exp7_swin_large_patch4_window12_384_in22k': 0.07210228959069125,
 #'exp7_vit_base_patch16_224_miil_in21k': 0.07195345528468275,
 #'exp7_vit_base_r50_s16_384': 0.058601333331186556,
 #'exp8_vit_base_patch16_224_miil_in21k': 0.05634679166402379,
 #'exp9_cait_m36_384': 0.0769990055041272
}


N_FOLDS = 5
all_preds = []

for path, w in tqdm(model_weights.items()):
    model_name = path.split('_', 1)[-1]
    try:
        img_size = int(model_name.split('_')[-1])
    except:
        try:
            img_size = int(model_name.split('_')[-2])
        except:
            img_size = 224
    
    def proc(pred):
        return pred
    num_classes = 1
    if 'exp4_' in path:
        model_dir = '../input/model-exp4-0106/pet'
        loss = BCEWithLogitsLossFlat()
    elif 'exp7_' in path:
        model_dir = '../input/model-exp7-0106/pet'
        loss = BCEWithLogitsLossFlat()
    elif 'exp8_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet'
        loss = MSELossFlat()
    elif 'exp9_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet' 
        loss = CrossEntropyLossFlat()
        num_classes = 100
        def proc(pred):
            return (pred.argmax(axis=1) + 1) / 100
    elif 'exp15_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet'
        loss = MSELossFlat()
        def proc(pred):
            return (np.exp(pred) / 100).clip(0.01, 1)
    elif 'exp20_' in path:
        model_dir = '../input/model-exp20/pet/'
        loss = BCEWithLogitsLossFlat()
        img_size = 384
    else:
        raise
                 
    dls = get_data(img_size)
            
    for i in range(N_FOLDS):
        if 'convnext' in path:
            model = create_model(model_name, pretrained=False)
            model.head = nn.Linear(in_features=model.head.in_features, out_features=1, bias=True)
        else:
            model = create_model(model_name, pretrained=False, num_classes=num_classes)

        model.load_state_dict(torch.load(f'{model_dir}/{path}/{model_name}_{i}.pth'))

        learn = Learner(dls, model, 
                        loss_func=loss,
                        metrics=petfinder_rmse).to_fp16()

        test_dl = dls.test_dl(test_df)

        preds, _ = learn.get_preds(dl=test_dl)
        #preds, _ = learn.tta(dl=test_dl, n=2, beta=0)
        
        preds = proc(preds).flatten()
        
        all_preds.append(preds * w / N_FOLDS)

        del learn
        torch.cuda.empty_cache()
        gc.collect()
    del dls
    torch.cuda.empty_cache()
    gc.collect()

preds = np.sum(np.stack(all_preds), axis=0)

pawpularity = round(preds[0] * 100)

print(f'Pawpularity of your pet is {pawpularity}.')

In [ ]:
draw = ImageDraw.Draw(img)
font = ImageFont.truetype("../input/arial-font/arial.ttf", 48)
draw.text((0,0), f'Pawpularity: {pawpularity}', font=font)
img